In [16]:
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
import requests

In [17]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

results = search_tool.invoke("top news in india today ?")

In [18]:
results

"India News | Latest India News | Read latest and breaking news from India. Today's top India news headlines, news on Indian politics, elections, government, business, technology, and Bollywood. Read all India news and breaking news today from India on politics, business, entertainment, technology, sports, lifestyle and more at CNBC TV18. ... Top India News. Kerala sees earliest onset of Southwest Monsoon in 16 years; advances over Karnataka, Mahe. May 24, 2025 6:36 PM IST. Zee News brings latest news from India and World on breaking news, today news headlines, politics, business, technology, bollywood, entertainment, sports and others. Find exclusive news stories on Indian politics, current affairs, cricket matches, festivals and events Reuters.com is your online source for the latest India news stories and current events, ensuring our readers up to date with any breaking news developments CNN News18 Live TV: Stay informed with CNN News18 Live TV. Watch Live streaming online for break

In [19]:
@tool
def get_weather_data(city: str) -> str:
  """
  This function fetches the current weather data for a given city
  """
  url = f'https://api.weatherstack.com/current?access_key=4ad15bb60fcb40e71d31912be476ae11&query={city}'

  response = requests.get(url)

  return response.json()

In [20]:
llm = ChatOpenAI()

In [21]:
llm.invoke("hi") # testing

AIMessage(content='Hello! How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-71e2c81d-d1cc-463f-b41b-c04e856a47e7-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [22]:
# now create agent
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub # to import predefined prompts

In [23]:
# step 2: pull the react prompt from langchain hub
# will be using react(reasoning and action) design pattern 
# for creating the agent
prompt = hub.pull("hwchase17/react") # pulls the standard react agent prompt

In [24]:
# step 3: create the react agent manually with the pulled prompt
agent = create_react_agent(
    llm=llm,
    tools=[search_tool, get_weather_data],
    prompt=prompt
)

In [25]:
# step 4: wrap it with agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool, get_weather_data],
    verbose=True
)

In [26]:
# step 5: Invoke
response = agent_executor.invoke({"input": "Find the capital of Madhya Pardesh. then find its current weather condition."})
print(response)



> Entering new AgentExecutor chain...
I should first search for the capital of Madhya Pradesh and then use the get_weather_data function to find its current weather condition.
Action: duckduckgo_search
Action Input: "capital of Madhya Pradesh"Madhya Pradesh, state of India that is situated in the heart of the country. It has no coastline and no international frontier. Its physiography is characterized by low hills, extensive plateaus, and river valleys. The capital is Bhopal, in the west-central part of the state. Indore (/ ɪ n ˈ d ɔːr / ⓘ; ISO: Indaura, Hindi: [ɪn̪d̪ɔːr]) is the largest and most populous city in the Indian state of Madhya Pradesh. [15] It is the commercial hub of Madhya Pradesh. It is consistently ranked as the cleanest city in India. [16] It serves as the headquarters of both the Indore District and the Indore Division.It is also considered the state education hub and ... However, Madhya Bharat was a temporary entity, and the map of central India was soon to be red

In [27]:
response["output"]

'The capital of Madhya Pradesh is Bhopal and its current weather condition is partly cloudy with a temperature of 33°C.'